Kütüphanelerin Kurulumu ve Google Drive Bağlantısı

In [ ]:
!pip install transformers datasets torch torchvision Pillow accelerate -U

from google.colab import drive
drive.mount('/content/drive')

import os
import json
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from transformers import ViTFeatureExtractor, ViTModel, BertTokenizer, BertModel
from torch.optim import AdamW
from tqdm.notebook import tqdm
import numpy as np
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Kullanılan cihaz: {device}")


Mounted at /content/drive
Kullanılan cihaz: cuda


Sabitler ve Veri Yolları

In [ ]:
DRIVE_BASE_PATH = '/content/drive/MyDrive/DerinOgrenme/'

IMAGES_ZIP_PATH = os.path.join(DRIVE_BASE_PATH, 'images.zip')
CAPTIONS_JSON_PATH = os.path.join(DRIVE_BASE_PATH, 'captions.json')
LABELS_ZIP_PATH = os.path.join(DRIVE_BASE_PATH, 'labels.zip')
KEYPOINTS_ZIP_PATH = os.path.join(DRIVE_BASE_PATH, 'keypoints.zip')
SEGM_ZIP_PATH = os.path.join(DRIVE_BASE_PATH, 'segm.zip')
DENSEPOSE_ZIP_PATH = os.path.join(DRIVE_BASE_PATH, 'densepose.zip')

COLAB_WORKING_DIR = '/content/DeepFashionMultiModal_data/'
IMAGE_DIR_UNZIPPED = os.path.join(COLAB_WORKING_DIR, 'images')

os.makedirs(COLAB_WORKING_DIR, exist_ok=True)
os.makedirs(IMAGE_DIR_UNZIPPED, exist_ok=True)

IMG_MODEL_NAME = 'google/vit-base-patch16-224-in21k'
TEXT_MODEL_NAME = 'bert-base-uncased'
EMBEDDING_DIM = 512
TEMPERATURE = 0.07

BATCH_SIZE = 16
LEARNING_RATE = 1e-5
EPOCHS = 10


In [ ]:
import zipfile

def unzip_file(zip_path, extract_to_path):
    """Belirtilen zip dosyasını hedef yola çıkarır."""
    if not os.path.exists(zip_path):
        print(f"HATA: Zip dosyası bulunamadı: {zip_path}")
        return False
    try:
        with zipfile.ZipFile(zip_path, 'r') as zip_ref:
            print(f"'{os.path.basename(zip_path)}' dosyası '{extract_to_path}' klasörüne çıkarılıyor...")
            zip_ref.extractall(extract_to_path)
        print(f"Çıkarma işlemi tamamlandı: '{extract_to_path}'")
        return True
    except Exception as e:
        print(f"Zip dosyası çıkarılırken hata oluştu: {e}")
        return False

print("Görsel dosyaları işleniyor...")

if not os.listdir(IMAGE_DIR_UNZIPPED):
    if unzip_file(IMAGES_ZIP_PATH, IMAGE_DIR_UNZIPPED):
        print(f"Görseller başarıyla '{IMAGE_DIR_UNZIPPED}' klasörüne çıkarıldı.")
    else:
        print(f"Görseller çıkarılamadı. Lütfen '{IMAGES_ZIP_PATH}' dosyasını kontrol edin.")
else:
    print(f"'{IMAGE_DIR_UNZIPPED}' klasörü zaten dolu. Zip çıkarma işlemi atlandı.")



Görsel dosyaları işleniyor...
'images.zip' dosyası '/content/DeepFashionMultiModal_data/images' klasörüne çıkarılıyor...
Çıkarma işlemi tamamlandı: '/content/DeepFashionMultiModal_data/images'
Görseller başarıyla '/content/DeepFashionMultiModal_data/images' klasörüne çıkarıldı.


In [ ]:
IMAGE_DIR_UNZIPPED = '/content/DeepFashionMultiModal_data/images/images'


Veri Yükleme ve Özel Dataset Sınıfı

In [ ]:
def load_text_descriptions(json_file_path):
    if not os.path.exists(json_file_path):
        print(f"HATA: JSON dosyası bulunamadı: {json_file_path}")
        return None
    with open(json_file_path, 'r') as f:
        descriptions = json.load(f)
    if isinstance(descriptions, list) and descriptions and isinstance(descriptions[0], dict) and "image" in descriptions[0] and "caption" in descriptions[0]:
        print("JSON formatı: Liste içinde sözlükler. Dönüştürülüyor...")
        return {item["image"]: item["caption"] for item in descriptions}
    elif isinstance(descriptions, dict):
        print("JSON formatı: Sözlük (image_filename -> caption). Doğrudan kullanılıyor.")
        return descriptions
    else:
        print("HATA: captions.json dosyasının formatı anlaşılamadı. Lütfen yapıyı kontrol edin.")
        return None


class FashionMultiModalDataset(Dataset):
    def __init__(self, image_root_dir, text_descriptions_dict, img_processor, text_tokenizer, max_text_len=128):
        self.image_root_dir = image_root_dir
        self.text_descriptions_dict = text_descriptions_dict

        self.valid_items = []
        available_images = set(os.listdir(self.image_root_dir))

        for img_filename, description in self.text_descriptions_dict.items():
            if img_filename in available_images:
                self.valid_items.append({"image_filename": img_filename, "description": description})

        if not self.valid_items:
            raise ValueError("Açıklamalar ve görseller eşleştirildikten sonra geçerli öğe bulunamadı.")

        print(f"{len(self.valid_items)} adet geçerli görsel-metin çifti bulundu.")

        self.img_processor = img_processor
        self.text_tokenizer = text_tokenizer
        self.max_text_len = max_text_len

    def __len__(self):
        return len(self.valid_items)

    def __getitem__(self, idx):
        item = self.valid_items[idx]
        img_filename = item["image_filename"]
        text_desc = item["description"]

        img_path = os.path.join(self.image_root_dir, img_filename)

        try:
            image = Image.open(img_path).convert('RGB')
            image_processed = self.img_processor(images=image, return_tensors="pt")['pixel_values'].squeeze(0)
        except Exception as e:
            print(f"Görsel yüklenirken hata: {img_path}, Hata: {e}")
            return None

        text_tokenized = self.text_tokenizer(
            text_desc,
            padding='max_length',
            truncation=True,
            max_length=self.max_text_len,
            return_tensors="pt"
        )

        input_ids = text_tokenized['input_ids'].squeeze(0)
        attention_mask = text_tokenized['attention_mask'].squeeze(0)

        return {
            'pixel_values': image_processed,
            'input_ids': input_ids,
            'attention_mask': attention_mask
        }

img_processor = ViTFeatureExtractor.from_pretrained(IMG_MODEL_NAME)
text_tokenizer = BertTokenizer.from_pretrained(TEXT_MODEL_NAME)

print("Metin açıklamaları (captions.json) yükleniyor...")
all_text_descriptions = load_text_descriptions(CAPTIONS_JSON_PATH)

if all_text_descriptions:
    try:
        dataset_full = FashionMultiModalDataset(IMAGE_DIR_UNZIPPED, all_text_descriptions, img_processor, text_tokenizer)

        if len(dataset_full) >= 3:
            train_size = int(0.7 * len(dataset_full))
            val_size = int(0.15 * len(dataset_full))
            test_size = len(dataset_full) - train_size - val_size

            if test_size <= 0 and val_size > 1:
                test_size = max(1, int(0.05 * len(dataset_full)))
                val_size = len(dataset_full) - train_size - test_size
                if val_size <=0:
                    val_size = 0
                    test_size = len(dataset_full) - train_size
            elif test_size <= 0:
                 raise ValueError("Test seti için yeterli veri ayrılamadı. Veri setiniz çok küçük olabilir.")


            print(f"Veri Seti Boyutları: Toplam={len(dataset_full)}, Eğitim={train_size}, Doğrulama={val_size}, Test={test_size}")

            if train_size <=0 or (val_size < 0) or (test_size <0) or (train_size + val_size + test_size != len(dataset_full)):
                print("HATA: Veri seti bölme boyutları geçersiz. Lütfen kontrol edin.")
                print(f"train_size: {train_size}, val_size: {val_size}, test_size: {test_size}, toplam: {len(dataset_full)}")
                raise SystemExit("Veri seti bölme hatası.")

            train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(
                dataset_full, [train_size, val_size, test_size]
            )

            def collate_fn(batch):
                batch = list(filter(lambda x: x is not None, batch))
                if not batch: return None
                return torch.utils.data.dataloader.default_collate(batch)

            train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)

            if val_size > 0:
                val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
            else:
                val_dataloader = None
                print("Uyarı: Doğrulama seti oluşturulmadı (val_size=0).")

            if test_size > 0:
                test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
            else:
                test_dataloader = None
                print("Uyarı: Test seti oluşturulmadı (test_size=0).")

            print("Veri yükleyiciler (eğitim, doğrulama, test) hazır.")
        else:
            print("HATA: Dataset oluşturuldu ancak geçerli öğe bulunamadı veya test için yeterli değil.")
    except:
        print("HATA: Dataset oluşturulurken hata oluştu.")
else:
    print("HATA: Metin açıklamaları yüklenirken hata oluştu.")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/160 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/vit/feature_extraction_vit.py:30: FutureWarning: The class ViTFeatureExtractor is deprecated and will be removed in version 5 of Transformers. Please use ViTImageProcessor instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

Metin açıklamaları (captions.json) yükleniyor...
JSON formatı: Sözlük (image_filename -> caption). Doğrudan kullanılıyor.
42544 adet geçerli görsel-metin çifti bulundu.
Veri Seti Boyutları: Toplam=42544, Eğitim=29780, Doğrulama=6381, Test=6383
Veri yükleyiciler (eğitim, doğrulama, test) hazır.


Çok Modlu Modelin Tanımlanması

In [ ]:
class MultiModalFashionModel(nn.Module):
    def __init__(self, img_model_name, text_model_name, embedding_dim, temperature=0.07):
        super().__init__()
        self.image_encoder = ViTModel.from_pretrained(img_model_name)
        self.text_encoder = BertModel.from_pretrained(text_model_name)

        img_feature_dim = self.image_encoder.config.hidden_size
        text_feature_dim = self.text_encoder.config.hidden_size

        self.image_projection = nn.Linear(img_feature_dim, embedding_dim)
        self.text_projection = nn.Linear(text_feature_dim, embedding_dim)

        self.temperature = temperature
        self.logit_scale = nn.Parameter(torch.ones([]) * np.log(1 / temperature))


    def encode_image(self, pixel_values):
        outputs = self.image_encoder(pixel_values=pixel_values)
        image_features = outputs.pooler_output
        image_embedding = self.image_projection(image_features)
        return F.normalize(image_embedding, p=2, dim=-1)

    def encode_text(self, input_ids, attention_mask):
        outputs = self.text_encoder(input_ids=input_ids, attention_mask=attention_mask)
        text_features = outputs.pooler_output
        text_embedding = self.text_projection(text_features)
        return F.normalize(text_embedding, p=2, dim=-1)

    def forward(self, pixel_values, input_ids, attention_mask):
        image_embeddings = self.encode_image(pixel_values)
        text_embeddings = self.encode_text(input_ids, attention_mask)
        return image_embeddings, text_embeddings

    def contrastive_loss(self, image_embeddings, text_embeddings):

        logit_scale = self.logit_scale.exp()
        logits_per_image = logit_scale * image_embeddings @ text_embeddings.t()
        logits_per_text = logit_scale * text_embeddings @ image_embeddings.t()

        batch_size = image_embeddings.shape[0]
        labels = torch.arange(batch_size, device=image_embeddings.device)

        loss_img = F.cross_entropy(logits_per_image, labels)
        loss_text = F.cross_entropy(logits_per_text, labels)

        loss = (loss_img + loss_text) / 2.0
        return loss

model = MultiModalFashionModel(IMG_MODEL_NAME, TEXT_MODEL_NAME, EMBEDDING_DIM, TEMPERATURE).to(device)
print("Model başarıyla oluşturuldu ve cihaza taşındı.")


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/346M [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Model başarıyla oluşturuldu ve cihaza taşındı.


Eğitim Döngüsü

In [ ]:
optimizer = AdamW(model.parameters(), lr=LEARNING_RATE)

def train_epoch(model, dataloader, optimizer, device):
    model.train()
    total_loss = 0

    progress_bar = tqdm(dataloader, desc="Eğitim Adımı", leave=False)
    for batch in progress_bar:
        if batch is None: continue

        pixel_values = batch['pixel_values'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        optimizer.zero_grad()

        image_embeddings, text_embeddings = model(pixel_values, input_ids, attention_mask)
        loss = model.contrastive_loss(image_embeddings, text_embeddings)

        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        progress_bar.set_postfix({'loss': loss.item()})

    return total_loss / len(dataloader)

def validate_epoch(model, dataloader, device):
    if dataloader is None:
        print("Doğrulama dataloader'ı yok, doğrulama adımı atlanıyor.")
        return float('inf')

    model.eval()
    total_loss = 0
    with torch.no_grad():
        progress_bar = tqdm(dataloader, desc="Doğrulama Adımı", leave=False)
        for batch in progress_bar:
            if batch is None: continue

            pixel_values = batch['pixel_values'].to(device)
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)

            image_embeddings, text_embeddings = model(pixel_values, input_ids, attention_mask)
            loss = model.contrastive_loss(image_embeddings, text_embeddings)

            total_loss += loss.item()
            progress_bar.set_postfix({'val_loss': loss.item()})

    return total_loss / len(dataloader)

print("Eğitim başlıyor...")
best_val_loss = float('inf')
if 'train_dataloader' not in locals() or train_dataloader is None:
    print("HATA: Eğitim veri yükleyici (train_dataloader) tanımlanmamış veya boş. Lütfen Hücre 3'ü kontrol edin.")
else:
    for epoch in range(EPOCHS):
        print(f"\nEpoch {epoch+1}/{EPOCHS}")

        train_loss = train_epoch(model, train_dataloader, optimizer, device)
        print(f"Ortalama Eğitim Kaybı: {train_loss:.4f}")

        if 'val_dataloader' in locals() and val_dataloader is not None:
            val_loss = validate_epoch(model, val_dataloader, device)
            print(f"Ortalama Doğrulama Kaybı: {val_loss:.4f}")

            if val_loss < best_val_loss:
                best_val_loss = val_loss
                torch.save(model.state_dict(), 'best_multimodal_fashion_model.pth')
                print("En iyi model kaydedildi: best_multimodal_fashion_model.pth")
        else:
            print("Doğrulama yapılmıyor. Model her epoch sonunda kaydedilmiyor (sadece en iyi val_loss'a göre).")


print("Eğitim tamamlandı.")
if 'val_dataloader' not in locals() or val_dataloader is None:
     torch.save(model.state_dict(), 'final_multimodal_fashion_model_no_val.pth')
     print("Son model (doğrulama yapılmadı) kaydedildi: final_multimodal_fashion_model_no_val.pth")
     model_to_evaluate_path = 'final_multimodal_fashion_model_no_val.pth'
elif os.path.exists('best_multimodal_fashion_model.pth'):
    model_to_evaluate_path = 'best_multimodal_fashion_model.pth'
    print("En iyi doğrulanmış model test için kullanılacak.")
else:
    print("Kaydedilmiş bir model bulunamadı. Son epoch'taki model test edilecek (eğer varsa).")
    model_to_evaluate_path = None

if 'test_dataloader' in locals() and test_dataloader is not None:
    print("\nTest seti üzerinde son değerlendirme yapılıyor...")
    if model_to_evaluate_path and os.path.exists(model_to_evaluate_path):
        print(f"Test için model yükleniyor: {model_to_evaluate_path}")
        try:
            model.load_state_dict(torch.load(model_to_evaluate_path, map_location=device))
            print("Model başarıyla yüklendi.")
        except Exception as e:
            print(f"Model yüklenirken hata: {e}. Mevcut model durumuyla devam edilecek (eğer varsa).")
    elif 'model' not in locals() or model is None :
         print("Test için yüklenecek bir model veya eğitilmiş bir model nesnesi bulunamadı.")
    else:
        print("Kaydedilmiş model bulunamadı. Eğitimden sonraki son model durumuyla test ediliyor.")

    if 'model' in locals() and model is not None:
        final_scores = evaluate_model(model, test_dataloader, device)
        if final_scores:
            print("\nDeğerlendirme tamamlandı.")
    else:
        print("Değerlendirme yapılamadı: Model nesnesi mevcut değil.")
else:
    print("Test veri yükleyici (test_dataloader) bulunamadığından son değerlendirme yapılamadı.")


Eğitim başlıyor...

Epoch 1/10


Eğitim Adımı:   0%|          | 0/1862 [00:00<?, ?it/s]

Ortalama Eğitim Kaybı: 0.9512


Doğrulama Adımı:   0%|          | 0/399 [00:00<?, ?it/s]

Ortalama Doğrulama Kaybı: 0.4757
En iyi model kaydedildi: best_multimodal_fashion_model.pth

Epoch 2/10


...

Eğitim Adımı:   0%|          | 0/1862 [00:00<?, ?it/s]

Ortalama Eğitim Kaybı: 0.2345


Doğrulama Adımı:   0%|          | 0/399 [00:00<?, ?it/s]

Ortalama Doğrulama Kaybı: 0.2582
En iyi model kaydedildi: best_multimodal_fashion_model.pth

Epoch 4/10


Eğitim Adımı:   0%|          | 0/1862 [00:00<?, ?it/s]

In [ ]:
@torch.no_grad()
def get_all_embeddings(model, dataloader, device):
    model.eval()
    all_image_embeddings = []
    all_text_embeddings = []

    print("Test seti için tüm embedding'ler çıkarılıyor...")
    for batch in tqdm(dataloader, desc="Embedding Çıkarma (Test)"):
        if batch is None: continue

        pixel_values = batch['pixel_values'].to(device)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        image_embeddings, text_embeddings = model(pixel_values, input_ids, attention_mask)

        all_image_embeddings.append(image_embeddings.cpu())
        all_text_embeddings.append(text_embeddings.cpu())

    if not all_image_embeddings or not all_text_embeddings:
        print("Hata: Embedding listeleri boş. Dataloader'da sorun olabilir.")
        return None, None

    all_image_embeddings = torch.cat(all_image_embeddings, dim=0)
    all_text_embeddings = torch.cat(all_text_embeddings, dim=0)

    return all_image_embeddings, all_text_embeddings

def calculate_retrieval_metrics(image_embeddings, text_embeddings, k_values=[1, 5, 10]):
    num_samples = image_embeddings.shape[0]
    if num_samples == 0:
        return {f"R@{k} (I2T)": 0.0 for k in k_values}, {f"R@{k} (T2I)": 0.0 for k in k_values}

    sim_i2t = image_embeddings @ text_embeddings.t()

    i2t_metrics = {}
    for k in k_values:
        correct_at_k = 0
        for i in range(num_samples):
            query_sim = sim_i2t[i]
            _, top_k_indices = torch.topk(query_sim, k=k, largest=True)
            if i in top_k_indices:
                correct_at_k += 1
        i2t_metrics[f"R@{k} (I2T)"] = correct_at_k / num_samples

    sim_t2i = text_embeddings @ image_embeddings.t()

    t2i_metrics = {}
    for k in k_values:
        correct_at_k = 0
        for i in range(num_samples):
            query_sim = sim_t2i[i]
            _, top_k_indices = torch.topk(query_sim, k=k, largest=True)
            if i in top_k_indices:
                correct_at_k += 1
        t2i_metrics[f"R@{k} (T2I)"] = correct_at_k / num_samples

    return i2t_metrics, t2i_metrics

def evaluate_model(model, test_dataloader, device, k_values=[1, 5, 10]):
    if test_dataloader is None:
        print("Test dataloader'ı yok. Değerlendirme atlanıyor.")
        return None

    image_embeddings, text_embeddings = get_all_embeddings(model, test_dataloader, device)

    if image_embeddings is None or text_embeddings is None or image_embeddings.shape[0] == 0:
        print("Değerlendirme için yeterli embedding çıkarılamadı.")
        return None

    if image_embeddings.shape[0] != text_embeddings.shape[0]:
        print(f"Hata: Görüntü ({image_embeddings.shape[0]}) ve metin ({text_embeddings.shape[0]}) embedding sayıları eşleşmiyor.")
        return None

    print(f"\n{image_embeddings.shape[0]} örnek üzerinde geri getirme metrikleri hesaplanıyor...")
    i2t_scores, t2i_scores = calculate_retrieval_metrics(image_embeddings, text_embeddings, k_values)

    print("\n--- Görüntüden Metne Geri Getirme (I2T) Skorları ---")
    for metric, score in i2t_scores.items():
        print(f"{metric}: {score:.4f}")

    print("\n--- Metinden Görüntüye Geri Getirme (T2I) Skorları ---")
    for metric, score in t2i_scores.items():
        print(f"{metric}: {score:.4f}")

    return {"i2t": i2t_scores, "t2i": t2i_scores}


Modelin Yüklenmesi ve Embedding Çıkarma (Inference Örneği)

In [ ]:

def get_image_embedding(model, image_path, img_processor, device):
    model.eval()
    try:
        image = Image.open(image_path).convert('RGB')
        image_processed = img_processor(images=image, return_tensors="pt")['pixel_values'].to(device)
        with torch.no_grad():
            image_embedding = model.encode_image(image_processed)
        return image_embedding.cpu().numpy()
    except Exception as e:
        print(f"Görsel embedding'i çıkarılırken hata: {image_path}, Hata: {e}")
        return None

def get_text_embedding(model, text_description, text_tokenizer, device, max_text_len=128):
    model.eval()
    text_tokenized = text_tokenizer(
        text_description,
        padding='max_length',
        truncation=True,
        max_length=max_text_len,
        return_tensors="pt"
    )
    input_ids = text_tokenized['input_ids'].to(device)
    attention_mask = text_tokenized['attention_mask'].to(device)
    with torch.no_grad():
        text_embedding = model.encode_text(input_ids, attention_mask)
    return text_embedding.cpu().numpy()

if 'val_dataset' in locals() and len(val_dataset) > 0 :
    try:
        sample_idx = np.random.randint(0, len(val_dataset.dataset.image_filenames))
        sample_img_filename = val_dataset.dataset.image_filenames[sample_idx]
        sample_img_path = os.path.join(IMAGE_DIR, sample_img_filename)
        sample_text_desc = val_dataset.dataset.text_descriptions.get(sample_img_filename, "Açıklama bulunamadı.")

        print(f"\nÖrnek Görsel: {sample_img_path}")
        print(f"Örnek Metin: {sample_text_desc}")

        img_emb = get_image_embedding(model, sample_img_path, img_processor, device)
        text_emb = get_text_embedding(model, sample_text_desc, text_tokenizer, device)

        if img_emb is not None and text_emb is not None:
            print("Görsel Embedding Boyutu:", img_emb.shape)
            print("Metin Embedding Boyutu:", text_emb.shape)

            similarity = (img_emb @ text_emb.T).item() / (np.linalg.norm(img_emb) * np.linalg.norm(text_emb))
            print(f"Örnek görsel ve metin arası cosine benzerliği: {similarity:.4f}")
    except Exception as e:
        print(f"Inference örneği çalıştırılırken hata: {e}")
else:
    print("Inference örneği için val_dataset bulunamadı veya boş.")


Basit Tavsiye Sistemi (Kavramsal)

In [ ]:


def get_image_embedding(model, image_path, img_processor, device):
    model.eval()
    try:
        image = Image.open(image_path).convert('RGB')
        image_processed = img_processor(images=image, return_tensors="pt")['pixel_values'].to(device)
        with torch.no_grad():
            image_embedding = model.encode_image(image_processed)
        return image_embedding.cpu().numpy()
    except Exception as e:
        print(f"Görsel embedding'i çıkarılırken hata: {image_path}, Hata: {e}")
        return None

def get_text_embedding(model, text_description, text_tokenizer, device, max_text_len=128):
    model.eval()
    text_tokenized = text_tokenizer(
        text_description,
        padding='max_length',
        truncation=True,
        max_length=max_text_len,
        return_tensors="pt"
    )
    input_ids = text_tokenized['input_ids'].to(device)
    attention_mask = text_tokenized['attention_mask'].to(device)
    with torch.no_grad():
        text_embedding = model.encode_text(input_ids, attention_mask)
    return text_embedding.cpu().numpy()

if 'dataset' in locals() and len(dataset.valid_items) > 0 :
    try:
        sample_item_info = dataset.valid_items[np.random.randint(0, len(dataset.valid_items))]
        sample_img_filename = sample_item_info["image_filename"]
        sample_text_desc = sample_item_info["description"]
        sample_img_path = os.path.join(IMAGE_DIR_UNZIPPED, sample_img_filename)

        print(f"\nÖrnek Görsel: {sample_img_path}")
        print(f"Örnek Metin: {sample_text_desc}")

        img_emb = get_image_embedding(model, sample_img_path, img_processor, device)
        text_emb = get_text_embedding(model, sample_text_desc, text_tokenizer, device)

        if img_emb is not None and text_emb is not None:
            print("Görsel Embedding Boyutu:", img_emb.shape)
            print("Metin Embedding Boyutu:", text_emb.shape)

            similarity = (img_emb @ text_emb.T).item() / (np.linalg.norm(img_emb) * np.linalg.norm(text_emb))
            print(f"Örnek görsel ve metin arası cosine benzerliği: {similarity:.4f}")
    except Exception as e:
        print(f"Inference örneği çalıştırılırken hata: {e}")
else:
    print("Inference örneği için 'dataset' bulunamadı veya geçerli öğe içermiyor.")


Makale Kıyaslama

In [ ]:

@torch.no_grad()
def evaluate_classification_tasks(model, dataloader, device, num_categories, num_attributes, top_k_category=[1, 3, 5]):
    model.eval()

    total_category_correct = {k: 0 for k in top_k_category}
    total_category_samples = 0





    if not hasattr(model, 'predict_category') or not hasattr(model, 'predict_attributes'):
        print("HATA: Modelde 'predict_category' veya 'predict_attributes' metotları bulunmuyor.")
        print("Lütfen modelinizi bu tahmin görevleri için güncelleyin.")
        return None, None

    all_category_preds = []
    all_category_labels = []
    all_attribute_preds_probs = []
    all_attribute_labels = []

    print("Kategori ve Öznitelik tahmini için test seti değerlendiriliyor...")
    for batch in tqdm(dataloader, desc="Kategori/Öznitelik Değerlendirme"):
        if batch is None: continue

        pixel_values = batch['pixel_values'].to(device)
        if 'category_label' not in batch or 'attribute_labels' not in batch:
            print("HATA: Batch'te 'category_label' veya 'attribute_labels' bulunmuyor.")
            continue

        category_labels_batch = batch['category_label'].to(device)
        attribute_labels_batch = batch['attribute_labels'].to(device, dtype=torch.float)

        category_logits = model.predict_category(pixel_values)
        _, top_k_preds_category = torch.topk(category_logits, max(top_k_category), dim=1)

        total_category_samples += category_labels_batch.size(0)
        for k in top_k_category:
            corrects = top_k_preds_category[:, :k] == category_labels_batch.unsqueeze(1)
            total_category_correct[k] += corrects.any(dim=1).sum().item()

        attribute_logits = model.predict_attributes(pixel_values)
        attribute_probs = torch.sigmoid(attribute_logits)

        all_attribute_preds_probs.append(attribute_probs.cpu())
        all_attribute_labels.append(attribute_labels_batch.cpu())

    category_accuracies = {f"Top-{k} Acc (Category)": total_category_correct[k] / total_category_samples if total_category_samples > 0 else 0
                           for k in top_k_category}

    attribute_metrics = {}
    if all_attribute_labels and all_attribute_preds_probs:
        all_attribute_labels_cat = torch.cat(all_attribute_labels, dim=0)
        all_attribute_preds_probs_cat = torch.cat(all_attribute_preds_probs, dim=0)

        k_attr_values = [1, 3, 5]
        for k_attr in k_attr_values:
            recalls_at_k_attr = []
            for i in range(all_attribute_preds_probs_cat.shape[0]):
                sample_probs = all_attribute_preds_probs_cat[i]
                sample_labels = all_attribute_labels_cat[i]

                num_positive_labels = sample_labels.sum().item()
                if num_positive_labels == 0:
                    continue

                _, top_k_attr_indices = torch.topk(sample_probs, k=k_attr)

                retrieved_labels_in_top_k = sample_labels[top_k_attr_indices]

                true_positives_in_top_k = retrieved_labels_in_top_k.sum().item()

                recall_for_sample = true_positives_in_top_k / num_positive_labels if num_positive_labels > 0 else 0.0
                recalls_at_k_attr.append(recall_for_sample)

            if recalls_at_k_attr:
                 attribute_metrics[f"Avg Recall@{k_attr} (Attributes)"] = np.mean(recalls_at_k_attr)
            else:
                 attribute_metrics[f"Avg Recall@{k_attr} (Attributes)"] = 0.0

    return category_accuracies, attribute_metrics




NUM_CATEGORIES_IN_DATASET = 50
NUM_ATTRIBUTES_IN_DATASET = 463



if 'test_dataloader' in locals() and test_dataloader is not None and \
   'model' in locals() and model is not None and \
   hasattr(model, 'predict_category') and hasattr(model, 'predict_attributes'):

    print("\nKategori ve Öznitelik Tahmini için Son Değerlendirme:")
    category_scores, attribute_scores = evaluate_classification_tasks(
        model,
        test_dataloader,
        device,
        num_categories=NUM_CATEGORIES_IN_DATASET,
        num_attributes=NUM_ATTRIBUTES_IN_DATASET,
        top_k_category=[1, 3, 5]
    )

    if category_scores and attribute_scores:
        print("\n--- Kategori Tahmini Sonuçları (Liu et al. ile Karşılaştırma İçin) ---")
        for metric, score in category_scores.items():
            print(f"{metric}: {score:.4f}")

        print("\n--- Öznitelik Tahmini Sonuçları (Liu et al. ile Karşılaştırma İçin) ---")
        for metric, score in attribute_scores.items():
            print(f"{metric}: {score:.4f}")

        print("\nLiu et al. (CVPR 2016) Makalesi Tablo 2 (Category) ve Tablo 3 (Attribute) ile karşılaştırın.")
        print("Not: Veri seti bölünmeleri, öznitelik tanımları ve eğitim prosedürleri farklılık gösterebilir.")
    else:
        print("Kategori/Öznitelik değerlendirmesi yapılamadı.")
elif 'model' in locals() and model is not None and (not hasattr(model, 'predict_category') or not hasattr(model, 'predict_attributes')):
    print("UYARI: 'model' nesnesinde kategori/öznitelik tahmin metotları ('predict_category', 'predict_attributes') bulunmuyor.")
    print("Lütfen modelinizi bu görevler için güncelleyin ve eğitin.")
else:
    print("Kategori/Öznitelik değerlendirmesi için test_dataloader veya model bulunamadı/uygun değil.")
